# HW 6

## Imports

In [ ]:
import nbtools
nbtools.setup_nb()

In [ ]:
import itertools
from astropy import units, constants
import sympy
import pandas
from scipy import integrate
import numpy
import plotly.express as px
from plotly import graph_objects as go

from sympy.diffgeom import Manifold, Patch
from pystein import coords, metric, curvature, geodesic
from pystein.utilities import tensor_pow as tpow, full_simplify, boundary_filter

## Utilities

In [ ]:
import plotly.colors

def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    if intermed <= 0 or len(colorscale) == 1:
        return colorscale[0][1]
    if intermed >= 1:
        return colorscale[-1][1]

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    # noinspection PyUnboundLocalVariable
    return plotly.colors.find_intermediate_color(
        lowcolor=low_color, highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb")

## Theory

### A2

In [ ]:
M = Manifold('M', dim=4)
P = Patch('origin', M)

t, r, theta, phi, k = sympy.symbols('t r theta phi k')
cs = coords.CoordSystem('spherical', P, [t, r, theta, phi])
dt, dr, dtheta, dphi = cs.base_oneforms()
a = sympy.Function('a')(t)


ds2 = - tpow(dt, 2) + a ** 2 * ((1 / (1 - k * r ** 2)) * tpow(dr, 2) + 
                                r**2 * tpow(dtheta, 2) + 
                                r ** 2 * sympy.sin(theta) ** 2 * tpow(dphi, 2))
g = metric.Metric(twoform=ds2)
g

In [ ]:
G_rr = curvature.einstein_tensor_component(1, 1, g)

In [ ]:
sympy.simplify(G_rr.doit())

## Exercises B

### B3

In [ ]:
M = Manifold('M', dim=4)
P = Patch('origin', M)

t, r, theta, phi, k = sympy.symbols('t r theta phi k')
cs = coords.CoordSystem('spherical', P, [t, r, theta, phi])
dt, dr, dtheta, dphi = cs.base_oneforms()
a = sympy.Function('a')(t)


ds2 = - tpow(dt, 2) + a ** 2 * ((1 / (1 - k * r ** 2)) * tpow(dr, 2) + 
                                r**2 * tpow(dtheta, 2) + 
                                r ** 2 * sympy.sin(theta) ** 2 * tpow(dphi, 2))
g = metric.Metric(twoform=ds2)
g

In [ ]:
christoffels = []
for m, n, l in itertools.product(range(4), range(4), range(4)):
#     print(m, n, l)
    gamma = sympy.simplify(curvature.christoffel_symbol_component(l, m, n, metric=g).doit())
    christoffels.append(((m, n, l), gamma))
christoffels = [x for x in christoffels if x[1]]

In [ ]:
_ch = [((_mu, _nu, _rho), expr) for (_mu, _nu, _rho), expr in christoffels if _rho == 0]

In [ ]:
curvature.display_components(_ch)

In [ ]:
_exs = [expr for (_mu, _nu, _rho), expr in _ch]
_g_ii = [g.inverse.matrix[i, i] for i in range(1, 4)]

_x = sum([_gii * _e for _gii, _e in zip(_g_ii, _exs)])

In [ ]:
sympy.simplify(_x)

In [ ]:
sympy.simplify(_x)

In [ ]:
def covariant_deriv(mu, nu, vector, metric):
    symbols = metric.coord_system.base_symbols()
    
    x_mu = symbols[mu]
    v_nu = vector[nu]
    
    res = sympy.Derivative(v_nu, x_mu)
    
    for alpha in range(4):
        v_alpha = vector[alpha]
        gamma_mu_alpha_nu = sympy.simplify(curvature.christoffel_symbol_component(nu, mu, alpha, metric=g).doit())
        res += gamma_mu_alpha_nu * v_alpha

    res = sympy.simplify(res.doit())
        
    return res

In [ ]:
eta_vector = [
    0, 
    sympy.sqrt(1 - k * r**2) * sympy.cos(theta), 
    - sympy.sqrt(1 - k * r**2) * sympy.sin(theta) / r, 
    0,
]

In [ ]:
def cov_deriv_matrix(vector, g):
    components = [4 * [None] for i in range(4)]
    for mu in range(4):
        for nu in range(4):
            del_mu_v_nu = covariant_deriv(mu, nu, vector, g)
            components[mu][nu] = del_mu_v_nu
    return sympy.Matrix(components)

In [ ]:
cmat = cov_deriv_matrix(eta_vector, g)

In [ ]:
subs = [
    (sympy.Derivative(a, t), sympy.symbols('\dot{a}')),
    (a, sympy.symbols('a')),
]

In [ ]:
print((cmat * g.matrix).subs(subs)._repr_latex_())

In [ ]:
G_tt = curvature.einstein_tensor_component(0, 0, g).doit()

In [ ]:
full_simplify(G_tt)

In [ ]:
R = curvature.ricci_scalar(g).doit()

In [ ]:
full_simplify(R)

### B4

In [ ]:
ts = numpy.arange(0, 10, 0.1)
consts = numpy.arange(1, 5 + 1, .1)
c_norm = lambda c: (c - 1) / 4.0 

In [ ]:
def psi(t, a_o):
    return (2 / a_o) * numpy.sqrt(t)

In [ ]:
viridis_colors, _ = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Plasma)
colorscale = plotly.colors.make_colorscale(viridis_colors)

In [ ]:
fig = go.Figure()
for c in consts:
    color = get_continuous_color(colorscale, intermed=c_norm(c))
    fig.add_trace(go.Scatter(x=psi(ts, c), y=ts, line=dict(color=color, dash='solid')))
    fig.add_trace(go.Scatter(x=-psi(ts, c), y=ts, line=dict(color=color, dash='dot')))

fig.update_layout(
#     yaxis_range=[0,10],
#                   xaxis_range=[0,10],
                  width=700,
                  height=700,
                  showlegend=False,
                  title_text=r'$\text{Null Radial Geodesics in FLRW}(t, \psi)$', 
                  title_x=0.5,
                  xaxis_title=r'$\psi$',
                  yaxis_title=r'$t/t_o$')
fig.show()

In [ ]:
# fig.write_image('/Users/jim/repos/tex/homework/courses/PHYS510 - GR I/hw6/figures/fig-b4.pdf')

### B6

In [ ]:
ts = numpy.arange(0.001, 2.001, 0.001)
a_s = numpy.sqrt(ts)
H_s = 1 / (2 * ts)
R_s =  - 1 / (4 * ts ** 2)
rho_s = 3 / (8 * numpy.pi * ts ** 2)

In [ ]:
fig = go.Figure()


fig.add_trace(go.Scatter(x=ts, y=a_s, name=r'$a(t)$', line=dict(color='green', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=H_s, name=r'$H(t)$', line=dict(color='blue', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=R_s, name=r'$R_{ij}(t)$', line=dict(color='magenta', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=rho_s, name=r'$\rho(t)$', line=dict(color='purple', dash='solid')))


fig.update_layout(
    yaxis_range=[-3,3],
                  xaxis_range=[0,1],
                  width=700,
                  height=700,
                  showlegend=True,
                  title_text=r'$\text{Radiation-Dominated FLRW Evolution}$', 
                  title_x=0.5,
                  xaxis_title=r'$t/t_o$',
                  yaxis_title=r'$\text{Cosmological Quantity}$')
fig.show()

In [ ]:
# fig.write_image('/Users/jim/repos/tex/homework/courses/PHYS510 - GR I/hw6/figures/fig-b6.pdf')

### B7

In [ ]:
ts = numpy.arange(0.001, 2.001, 0.001)
a_s = ts ** (2.0 / 3.0)
H_s = 2 / (3 * ts)
R_s =  - 2 / (9 * ts ** 2)
rho_s = 1 / (2 * numpy.pi * ts ** 2)

In [ ]:
fig = go.Figure()


fig.add_trace(go.Scatter(x=ts, y=a_s, name=r'$a(t)$', line=dict(color='green', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=H_s, name=r'$H(t)$', line=dict(color='blue', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=R_s, name=r'$R_{ij}(t)$', line=dict(color='magenta', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=rho_s, name=r'$\rho(t)$', line=dict(color='purple', dash='solid')))


fig.update_layout(
    yaxis_range=[-3,3],
                  xaxis_range=[0,1],
                  width=700,
                  height=700,
                  showlegend=True,
                  title_text=r'$\text{Matter-Dominated FLRW Evolution}$', 
                  title_x=0.5,
                  xaxis_title=r'$t/t_o$',
                  yaxis_title=r'$\text{Cosmological Quantity}$')
fig.show()

In [ ]:
fig.write_image('/Users/jim/repos/tex/homework/courses/PHYS510 - GR I/hw6/figures/fig-b7.pdf')

### B8

In [ ]:
G, a_o, Lam, rho_o = sympy.symbols('G a_o Lambda rho_o')

In [ ]:
a_b8 = a_o * (sympy.sqrt(8 * sympy.pi * G * rho_o / Lam) *
              sympy.sinh(sympy.sqrt(3 * Lam) * t / 2)) ** (sympy.Rational(2, 3))

In [ ]:
a_b8

In [ ]:
a_dot_b8 = sympy.simplify(sympy.Derivative(a_b8, t).doit())

In [ ]:
a_dot_b8

In [ ]:
hubble_b8 = sympy.simplify(a_dot_b8 / a_b8)

In [ ]:
hubble_b8

In [ ]:
a_ddot_b8 = sympy.Derivative(a_dot_b8, t).doit()

In [ ]:
R_ij_b8 = sympy.simplify(a_ddot_b8 / a_b8)

In [ ]:
R_ij_b8

In [ ]:
rho_b8 = sympy.simplify(rho_o * (a_o / a_b8) ** 3)

In [ ]:
rho_b8

In [ ]:
lhs = sympy.simplify(a_dot_b8 ** 2)
lhs

In [ ]:
rhs = sympy.Rational(1, 3) * Lam * a_b8 ** 2 + sympy.Rational(8, 3) * sympy.pi * G * rho_o * (a_o ** 3) / a_b8
rhs = sympy.simplify(rhs)
rhs

In [ ]:
print(R_ij_b8._repr_latex_())

In [ ]:
sympy.simplify(lhs - rhs)

In [ ]:
rho_b8._

#### Visual

In [ ]:
a_b8

In [ ]:
a_b8_callable = sympy.lambdify((t, G, rho_o, Lam, a_o), a_b8)
H_b8_callable = sympy.lambdify((t, G, rho_o, Lam, a_o), hubble_b8)
R_b8_callable = sympy.lambdify((t, G, rho_o, Lam, a_o), R_ij_b8)
rho_b8_callable = sympy.lambdify((t, G, rho_o, Lam, a_o), rho_b8)

In [ ]:
a_b8_res = a_b8_callable(ts, 1, 1, 1, 1)
H_b8_res = H_b8_callable(ts, 1, 1, 1, 1)
R_b8_res = R_b8_callable(ts, 1, 1, 1, 1)
rho_b8_res = rho_b8_callable(ts, 1, 1, 1, 1)

In [ ]:
fig = go.Figure()


fig.add_trace(go.Scatter(x=ts, y=a_b8_res, name=r'$a(t)$', line=dict(color='green', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=H_b8_res, name=r'$H(t)$', line=dict(color='blue', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=R_b8_res, name=r'$R_{ij}(t)$', line=dict(color='magenta', dash='solid')))

fig.add_trace(go.Scatter(x=ts, y=rho_b8_res, name=r'$\rho(t)$', line=dict(color='purple', dash='solid')))


fig.update_layout(
    yaxis_range=[-3,3],
                  xaxis_range=[0,1],
                  width=700,
                  height=700,
                  showlegend=True,
                  title_text=r'$\text{Matter + }\Lambda\text{ Dominated FLRW Evolution}$', 
                  title_x=0.5,
                  xaxis_title=r'$t/t_o$',
                  yaxis_title=r'$\text{Cosmological Quantity}$')
fig.show()

In [ ]:
fig.write_image('/Users/jim/repos/tex/homework/courses/PHYS510 - GR I/hw6/figures/fig-b8.pdf')

## Problem C

### C1

In [ ]:
H = 70e3 * (units.m / (units.s * units.Mpc) )
H

In [ ]:
rho_crit = (3 / (8 * numpy.pi * constants.G)) * H ** 2
rho_crit

In [ ]:
rho_crit_per_proton = rho_crit / constants.m_p
rho_crit_per_proton

In [ ]:
rho_crit_per_proton.to((1 / units.m) ** 3)

In [ ]:
constants.G

### C2

In [ ]:
(1 / H).to(units.yr)

### C3

In [ ]:
omega_m = 0.31

In [ ]:
rho_crit = (3 / (8 * numpy.pi * constants.G)) * H ** 2
rho_crit = rho_crit.to(units.kg / units.m ** 3)
rho_init = omega_m * rho_crit

In [ ]:

sinh_arg = numpy.sqrt(rho_crit / rho_init * (1 - omega_m)).value

factor = 1 / numpy.sqrt(6 * numpy.pi * G * rho_crit * (1 - omega_m))
t_o = factor * numpy.arcsinh(sinh_arg)

In [ ]:
t_o.to(units.yr)

### C4

In [ ]:
omega_lam = 0.69
Lam_c4 = 8 * numpy.pi * constants.G * rho_crit * omega_lam
Lam_c4

In [ ]:
t_star = 2 * numpy.log(1 + numpy.sqrt(2)) / numpy.sqrt(3 * Lam_c4)
t_star

In [ ]:
t_star.to(units.yr)

In [ ]:
t_o.to(units.yr) - t_star.to(units.yr)